In [1]:
import numpy as np
import math
import pandas as pd
from numpy import nan
import tensorflow as tf

In [2]:
#从CSV文件中获取数据
print('--------load train & test file------')
# 获取训练数据
train_dataset = pd.read_csv('titanicData/train_titanic.csv')
print(train_dataset.shape)
train_dataset = train_dataset.as_matrix()

#获取测试数据
test_dataset = pd.read_csv('titanicData/test_titanic.csv')
print(test_dataset.shape)
test_dataset = test_dataset.as_matrix()

print("------finish loading --------------------")

--------load train & test file------
(891, 12)
(418, 11)
------finish loading --------------------


In [3]:
#打乱原有的数据集，避免多次训练过拟合
from sklearn.utils import shuffle
train_dataset = shuffle(train_dataset)
print ('finish shuffling the data')

# Holdout ( 2/3 to 1/3 )
num_features = train_dataset.shape[0] # number of features
print("Number of all features:", num_features)
split = int(num_features * 2/3)

train = train_dataset[:split]
test = train_dataset[split:]


finish shuffling the data
('Number of all features:', 891)


In [4]:
def average(data):
    print("data len:",len(data))
    sum = 0
    for i in range(len(data)):
        if(math.isnan(data[i])):
            continue
        sum += data[i]
    return sum/len(data)

In [5]:
#python切片操作,[x:y,m:n],表示矩阵x~y行，m~n列区间
#获取标签数据
train_labels = train[: ,1:2]
#获取图像像素数据
train_data = train[: ,2:]

test_labels = test[: ,1:2]
test_data = test[: ,2:]

print("train_labels:",train_labels.shape)
print("train_data:",train_data.shape)
print("test_labels:",test_labels.shape)
print("test_data:",test_data.shape)

('train_labels:', (594, 1))
('train_data:', (594, 10))
('test_labels:', (297, 1))
('test_data:', (297, 10))


In [6]:
def getData(train):
    average_age = average(train[:,3].reshape(-1))
    average_fare = average(train[:,7].reshape(-1))
    print("average age:",average_age)
    print("input:",train.shape)
    for index in range(train.shape[0]):
        #sex
        if(train[index,2] == 'male'):
            train[index,2] = 1
        elif(train[index,2] == 'female'):
            train[index,2] = 0
        else:
            train[index,2] = 2
    
        train[index,1] = 2
            
        #Embarked
        if(train[index,9] == 'C'):
            train[index,9] = 0
        elif(train[index,9] == 'Q'):
            train[index,9] = 1
        elif(train[index,9] == 'S'):
            train[index,9] = 2
        else:
            train[index,9] = 0

        #Age
        if(math.isnan(train[index,3]) or train[index,3] == ''):
            train[index,3] = average_age
#         if(train[index,3] <= 12):
#             train[index,3] = 0
#         elif(train[index,3] <= 40):
#             train[index,3] = 1
#         elif(train[index,3] <= 65):
#             train[index,3] = 2
#         else:
#             train[index,3] = 3

        #Fare
        if(math.isnan(train[index,7]) or train[index,7] == ''):
            train[index,7] = average_fare
#         if(train[index,7] <= 7):
#             train[index,7] = 0
#         elif(train[index,7] <= 20):
#             train[index,7] = 1
#         elif(train[index,7] <= 50):
#             train[index,7] = 2
#         else:
#             train[index,7] = 3
            
        #Cabin
        if(isinstance(train[index,8],str) == False):
            train[index,8] = 0
        elif('A' in train[index,8] or 'a' in train[index,8]):
            train[index,8] = 1
        elif('B' in train[index,8] or 'b' in train[index,8]):
            train[index,8] = 2
        elif('C' in train[index,8] or 'c' in train[index,8]):
            train[index,8] = 3
        elif('D' in train[index,8] or 'd' in train[index,8]):
            train[index,8] = 4
        elif('E' in train[index,8] or 'e' in train[index,8]):
            train[index,8] = 5
        elif('F' in train[index,8] or 'f' in train[index,8]):
            train[index,8] = 6
        else:
            train[index,8] = 0
#         print(train[index,8],"========",isinstance(train[index,8],str))
        

#     train = train[:,[0,2,3,4,5,7,8,9]]
    train = train[:,[0,1,2,3,4,5,7,8,9]]
    print("output:",train.shape)

    return train

In [7]:
#训练数据训练
from sklearn.ensemble import RandomForestClassifier

train_data = getData(train_data)
train_labels = train_labels[:,:].astype(int).ravel()
test_data = getData(test_data)
test_labels = test_labels[:,:].astype(int).ravel()
    
#RandomForestClassifier参数详解：http://blog.sina.com.cn/s/blog_a1f6697d0102w8ep.html
#rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=2, n_jobs=-1, random_state=42)
rnd_clf = RandomForestClassifier(criterion='gini', 
                             n_estimators=20,
                             min_samples_split=16,
                             min_samples_leaf=2,
                             max_features='auto',
                             oob_score=True,
                             n_jobs=-1,
                             random_state=50)

rnd_clf.fit(train_data, train_labels)
predict = rnd_clf.predict(test_data)

predict = predict.reshape(-1)
test_labels = test_labels.reshape(-1)

# 计算准确度
acc = np.mean(predict == test_labels) # calculate accuracy
print("The accuracy of the pure RandomForest classifier is:", acc)
    

('data len:', 594)
('data len:', 594)
('average age:', 24.386228956228955)
('input:', (594, 10))
('output:', (594, 9))
('data len:', 297)
('data len:', 297)
('average age:', 22.625420875420875)
('input:', (297, 10))
('output:', (297, 9))
('The accuracy of the pure RandomForest classifier is:', 0.85185185185185186)


In [8]:
test_data = getData(test_dataset[:,1:])
predict = rnd_clf.predict(test_data)
print(predict.shape)
print("predict:",predict)
print(test_dataset[:,0])
np.savetxt('titanicData/gender_submission.csv', np.c_[test_dataset[:,0],predict], delimiter=',', header = 'PassengerId,Survived', comments = '', fmt='%d')

('data len:', 418)
('data len:', 418)
('average age:', 24.044258373205743)
('input:', (418, 10))
('output:', (418, 9))
(418,)
('predict:', array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 